In [1]:
import keras
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model 
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras import backend as k 
from keras.callbacks import CSVLogger, ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.utils.vis_utils import plot_model
from keras.applications.vgg19 import decode_predictions
from keras.layers.normalization import BatchNormalization

from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import os

import tensorflow
from time import time

import pandas

import pickle as pk

from keras.wrappers.scikit_learn import KerasClassifier

seed = 7

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
np.random.seed(seed)

In [3]:
data_dir = '/data/oxford102/train'
img_width, img_height = 256, 256
batch_size = 32
epochs = 100
nr_categories = 102

In [4]:
img_width, img_height = 256, 256
train_data_dir = "/data/oxford102/train/"
validation_data_dir = "/data/oxford102/train/"
nb_train_samples = 4604
nb_validation_samples = 1094 

In [5]:
input_processor = keras.applications.vgg19.preprocess_input

In [6]:
train_val_datagen = ImageDataGenerator(preprocessing_function=input_processor, 
                                           validation_split=0.2)

train_val_datagen_aug = ImageDataGenerator(
        #rescale=1. / 255,
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest',
        preprocessing_function=input_processor,
        validation_split=0.2)

In [7]:

    
train_generator = train_val_datagen_aug.flow_from_directory(
        train_data_dir,  # this is the target directory
        target_size=(img_width, img_height),  # all images will be resized to 250x250
        batch_size=batch_size,
        subset="training",
        class_mode='categorical')

validation_generator = train_val_datagen_aug.flow_from_directory(
        train_data_dir,  # this is the target directory
        target_size=(img_width, img_height),  # all images will be resized to 250x250
        subset="validation",
        batch_size=batch_size,
        class_mode='categorical')


Found 4604 images belonging to 102 classes.
Found 1094 images belonging to 102 classes.


In [8]:
from keras import regularizers

k.set_learning_phase(0)

network_name = "vgg19"
img_width, img_height = (256, 256)
if network_name == "vgg16":
    base_model = keras.applications.vgg16.VGG16(weights = "imagenet", include_top=False, input_shape = (img_width, img_height, 3))
elif network_name == "vgg19":
    base_model = keras.applications.vgg19.VGG19(weights = "imagenet", include_top=False, input_shape = (img_width, img_height, 3))
else:
    raise Exception("check your network name")

for layer in base_model.layers[:]:
    layer.trainable = False

    #Adding custom Layers 
k.set_learning_phase(1)
x = base_model.output
x = Flatten()(x)
x = Dense(1024, activation="relu", 
          kernel_regularizer=regularizers.l2(0.01),
         #       activity_regularizer=regularizers.l1(0.001)
         )(x)
x = Dropout(0.5)(x)
x = BatchNormalization()(x, training=True)
#x = Dense(102, activation="relu")(x)
predictions = Dense(nr_categories, activation="softmax")(x)

_model = Model(input = base_model.input, output = predictions)

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:30: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


In [9]:
_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 256, 256, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 256, 256, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 256, 256, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 128, 128, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 128, 128, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 128, 128, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 64, 64, 128)       0         
__________

In [10]:
# _model.compile(loss = "categorical_crossentropy", optimizer = optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=None, decay=0.0), metrics=["accuracy"])


In [11]:
#run_training(model_name, _model, train_generator, validation_generator, params, num_train_img, num_val_img):
    

model_name = "vgg19"
num_train_img = 4604
num_val_img = 1094
params = {'network_name': 'vgg19',
         'image_aug': False,
         'optimizer': 'SGD',
          'optimizer_params': None, 
         'batch_size': 512,
         'epochs': 100,
         'image_size': (256, 256),
         'log_time': None}


np.random.seed(seed)
log_time = time()
params['log_time'] = log_time
batch_size = params.get("batch_size")

_model.compile(loss = "categorical_crossentropy", optimizer = optimizers.SGD(lr=0.001, momentum=0.9), metrics=["accuracy"])


base = '/data/oxford102/experiments'
path = os.path.join(base, str(log_time))
checkpoint = ModelCheckpoint(os.path.join(path, "{}_{}.h5".format(model_name, log_time)), monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=3, verbose=1, mode='auto')
tensorboard = TensorBoard(log_dir="logs/{}".format(log_time), histogram_freq=0, write_graph=True, write_images=True)
csv_logger = CSVLogger(os.path.join(path, "{}_{}.csv".format(model_name, log_time)), append=True, separator=';')

try:
    if not os.path.exists(path):
        os.makedirs(path)
    history_callback = _model.fit_generator(
            train_generator,
            steps_per_epoch=num_train_img // batch_size,
            epochs=params.get("epochs"),
            validation_data=validation_generator,
            validation_steps=num_val_img // batch_size,
            callbacks = [checkpoint, early, tensorboard, csv_logger])
except Exception as e:
    raise(e)
finally:
    pk.dump(params, open("experimental_params/experiments_{}.pk".format(log_time), "wb"), protocol=pk.HIGHEST_PROTOCOL)
    _model.save_weights(os.path.join(path, 'model_{}_weights_final_{}.h5'.format(model_name, log_time)))  # always save your weights after training or during training
    print(params)
    params

Epoch 1/100
8/8 [==============================] - 13s 2s/step - loss: 25.2712 - acc: 0.0078 - val_loss: 25.0739 - val_acc: 0.0000e+00

Epoch 00001: val_acc improved from -inf to 0.00000, saving model to /data/oxford102/experiments/1543117119.2325218/vgg19_1543117119.2325218.h5
Epoch 2/100
8/8 [==============================] - 6s 704ms/step - loss: 25.0496 - acc: 0.0234 - val_loss: 24.7788 - val_acc: 0.0469

Epoch 00002: val_acc improved from 0.00000 to 0.04688, saving model to /data/oxford102/experiments/1543117119.2325218/vgg19_1543117119.2325218.h5
Epoch 3/100
8/8 [==============================] - 6s 714ms/step - loss: 24.5847 - acc: 0.0547 - val_loss: 24.4382 - val_acc: 0.0781

Epoch 00003: val_acc improved from 0.04688 to 0.07812, saving model to /data/oxford102/experiments/1543117119.2325218/vgg19_1543117119.2325218.h5
Epoch 4/100
8/8 [==============================] - 9s 1s/step - loss: 24.3585 - acc: 0.0753 - val_loss: 24.3634 - val_acc: 0.1094

Epoch 00004: val_acc improved 

In [12]:
model_name = "vgg19"
num_train_img = 4604
num_val_img = 1094

params = {'network_name': 'vgg19',
         'image_aug': False,
         'optimizer': 'SGD',
          'optimizer_params': None, 
         'batch_size': 128,
         'epochs': 100,
         'image_size': (256, 256),
         'log_time': None}


np.random.seed(seed)
log_time = time()
params['log_time'] = log_time
batch_size = params.get("batch_size")

_model.compile(loss = "categorical_crossentropy", optimizer = optimizers.SGD(lr=0.001, momentum=0.9), metrics=["accuracy"])


base = '/data/oxford102/experiments'
path = os.path.join(base, str(log_time))
checkpoint = ModelCheckpoint(os.path.join(path, "{}_{}.h5".format(model_name, log_time)), monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=3, verbose=1, mode='auto')
tensorboard = TensorBoard(log_dir="logs/{}".format(log_time), histogram_freq=0, write_graph=True, write_images=True)
csv_logger = CSVLogger(os.path.join(path, "{}_{}.csv".format(model_name, log_time)), append=True, separator=';')

try:
    if not os.path.exists(path):
        os.makedirs(path)
    history_callback = _model.fit_generator(
            train_generator,
            steps_per_epoch=num_train_img // batch_size,
            epochs=params.get("epochs"),
            validation_data=validation_generator,
            validation_steps=num_val_img // batch_size,
            callbacks = [checkpoint, early, tensorboard, csv_logger])
except Exception as e:
    raise(e)
finally:
    pk.dump(params, open("experimental_params/experiments_{}.pk".format(log_time), "wb"), protocol=pk.HIGHEST_PROTOCOL)
    _model.save_weights(os.path.join(path, 'model_{}_weights_final_{}.h5'.format(model_name, log_time)))  # always save your weights after training or during training
    print(params)
    params

Epoch 1/100
35/35 [==============================] - 35s 1s/step - loss: 22.4890 - acc: 0.3179 - val_loss: 22.6014 - val_acc: 0.2930

Epoch 00001: val_acc improved from -inf to 0.29297, saving model to /data/oxford102/experiments/1543117217.7432935/vgg19_1543117217.7432935.h5
Epoch 2/100
35/35 [==============================] - 29s 834ms/step - loss: 22.1251 - acc: 0.3571 - val_loss: 22.2259 - val_acc: 0.2969

Epoch 00002: val_acc improved from 0.29297 to 0.29688, saving model to /data/oxford102/experiments/1543117217.7432935/vgg19_1543117217.7432935.h5
Epoch 3/100
35/35 [==============================] - 28s 796ms/step - loss: 21.8851 - acc: 0.3759 - val_loss: 21.8994 - val_acc: 0.3555

Epoch 00003: val_acc improved from 0.29688 to 0.35547, saving model to /data/oxford102/experiments/1543117217.7432935/vgg19_1543117217.7432935.h5
Epoch 4/100
35/35 [==============================] - 28s 791ms/step - loss: 21.6039 - acc: 0.4170 - val_loss: 21.5058 - val_acc: 0.4336

Epoch 00004: val_acc

In [13]:
model_name = "vgg19"
num_train_img = 4604
num_val_img = 1094
params = {'network_name': 'vgg19',
         'image_aug': False,
         'optimizer': 'SGD',
          'optimizer_params': None, 
         'batch_size': 64,
         'epochs': 100,
         'image_size': (256, 256),
         'log_time': None}


np.random.seed(seed)
log_time = time()
params['log_time'] = log_time
batch_size = params.get("batch_size")

_model.compile(loss = "categorical_crossentropy", optimizer = optimizers.SGD(lr=0.001, momentum=0.9), metrics=["accuracy"])


base = '/data/oxford102/experiments'
path = os.path.join(base, str(log_time))
checkpoint = ModelCheckpoint(os.path.join(path, "{}_{}.h5".format(model_name, log_time)), monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=3, verbose=1, mode='auto')
tensorboard = TensorBoard(log_dir="logs/{}".format(log_time), histogram_freq=0, write_graph=True, write_images=True)
csv_logger = CSVLogger(os.path.join(path, "{}_{}.csv".format(model_name, log_time)), append=True, separator=';')

try:
    if not os.path.exists(path):
        os.makedirs(path)
    history_callback = _model.fit_generator(
            train_generator,
            steps_per_epoch=num_train_img // batch_size,
            epochs=params.get("epochs"),
            validation_data=validation_generator,
            validation_steps=num_val_img // batch_size,
            callbacks = [checkpoint, early, tensorboard, csv_logger])
except Exception as e:
    raise(e)
finally:
    pk.dump(params, open("experimental_params/experiments_{}.pk".format(log_time), "wb"), protocol=pk.HIGHEST_PROTOCOL)
    _model.save_weights(os.path.join(path, 'model_{}_weights_final_{}.h5'.format(model_name, log_time)))  # always save your weights after training or during training
    print(params)
    params

Epoch 1/100
71/71 [==============================] - 71s 1s/step - loss: 19.4662 - acc: 0.5977 - val_loss: 19.4825 - val_acc: 0.5423

Epoch 00001: val_acc improved from -inf to 0.54228, saving model to /data/oxford102/experiments/1543117533.4054904/vgg19_1543117533.4054904.h5
Epoch 2/100
71/71 [==============================] - 61s 859ms/step - loss: 19.0476 - acc: 0.6107 - val_loss: 18.9521 - val_acc: 0.5827

Epoch 00002: val_acc improved from 0.54228 to 0.58272, saving model to /data/oxford102/experiments/1543117533.4054904/vgg19_1543117533.4054904.h5
Epoch 3/100
71/71 [==============================] - 70s 980ms/step - loss: 18.5245 - acc: 0.6488 - val_loss: 18.4785 - val_acc: 0.6048

Epoch 00003: val_acc improved from 0.58272 to 0.60478, saving model to /data/oxford102/experiments/1543117533.4054904/vgg19_1543117533.4054904.h5
Epoch 4/100
71/71 [==============================] - 61s 866ms/step - loss: 18.0958 - acc: 0.6530 - val_loss: 18.0835 - val_acc: 0.6066

Epoch 00004: val_acc

In [ ]:
model_name = "vgg19"
num_train_img = 4604
num_val_img = 1094
params = {'network_name': 'vgg19',
         'image_aug': False,
         'optimizer': 'SGD',
          'optimizer_params': None, 
         'batch_size': 32,
         'epochs': 100,
         'image_size': (256, 256),
         'log_time': None}


np.random.seed(seed)
log_time = time()
params['log_time'] = log_time
batch_size = params.get("batch_size")

_model.compile(loss = "categorical_crossentropy", optimizer = optimizers.SGD(lr=0.001, momentum=0.9), metrics=["accuracy"])


base = '/data/oxford102/experiments'
path = os.path.join(base, str(log_time))
checkpoint = ModelCheckpoint(os.path.join(path, "{}_{}.h5".format(model_name, log_time)), monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=3, verbose=1, mode='auto')
tensorboard = TensorBoard(log_dir="logs/{}".format(log_time), histogram_freq=0, write_graph=True, write_images=True)
csv_logger = CSVLogger(os.path.join(path, "{}_{}.csv".format(model_name, log_time)), append=True, separator=';')

try:
    if not os.path.exists(path):
        os.makedirs(path)
    history_callback = _model.fit_generator(
            train_generator,
            steps_per_epoch=num_train_img // batch_size,
            epochs=params.get("epochs"),
            validation_data=validation_generator,
            validation_steps=num_val_img // batch_size,
            callbacks = [checkpoint, early, tensorboard, csv_logger])
except Exception as e:
    raise(e)
finally:
    pk.dump(params, open("experimental_params/experiments_{}.pk".format(log_time), "wb"), protocol=pk.HIGHEST_PROTOCOL)
    _model.save_weights(os.path.join(path, 'model_{}_weights_final_{}.h5'.format(model_name, log_time)))  # always save your weights after training or during training
    print(params)
    params

Epoch 1/100
143/143 [==============================] - 143s 998ms/step - loss: 14.5837 - acc: 0.0446 - val_loss: 14.1087 - val_acc: 0.0533

Epoch 00001: val_acc improved from -inf to 0.05331, saving model to /data/oxford102/experiments/1543119372.975714/vgg19_1543119372.975714.h5
Epoch 2/100
  2/143 [..............................] - ETA: 1:48 - loss: 13.9183 - acc: 0.0625

# FINETUNE TOP LAYERS

In [ ]:
for i, layer in enumerate(base_model.layers[11:]):
    print(i, layer.name)
    layer.trainable = True

In [ ]:
model_name = "vgg19"
num_train_img = 4604
num_val_img = 1094
params = {'network_name': 'vgg19',
         'image_aug': False,
         'optimizer': 'SGD',
          'optimizer_params': None, 
         'batch_size': 64,
         'epochs': 100,
         'image_size': (256, 256),
         'log_time': None}


np.random.seed(seed)
log_time = time()
params['log_time'] = log_time
batch_size = params.get("batch_size")

_model.compile(loss = "categorical_crossentropy", optimizer = optimizers.SGD(lr=0.001, momentum=0.9), metrics=["accuracy"])


base = '/data/oxford102/experiments'
path = os.path.join(base, str(log_time))
checkpoint = ModelCheckpoint(os.path.join(path, "{}_{}.h5".format(model_name, log_time)), monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=3, verbose=1, mode='auto')
tensorboard = TensorBoard(log_dir="logs/{}".format(log_time), histogram_freq=0, write_graph=True, write_images=True)
csv_logger = CSVLogger(os.path.join(path, "{}_{}.csv".format(model_name, log_time)), append=True, separator=';')

try:
    if not os.path.exists(path):
        os.makedirs(path)
    history_callback = _model.fit_generator(
            train_generator,
            steps_per_epoch=num_train_img // batch_size,
            epochs=params.get("epochs"),
            validation_data=validation_generator,
            validation_steps=num_val_img // batch_size,
            callbacks = [checkpoint, early, tensorboard, csv_logger])
except Exception as e:
    raise(e)
finally:
    pk.dump(params, open("experimental_params/experiments_{}.pk".format(log_time), "wb"), protocol=pk.HIGHEST_PROTOCOL)
    _model.save_weights(os.path.join(path, 'model_{}_weights_final_{}.h5'.format(model_name, log_time)))  # always save your weights after training or during training
    print(params)
    params